In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynlo
from scipy import constants

In [ ]:
from dataclasses import dataclass

@dataclass
class NobleGasData:
    name: str
    beta2: float          # [fs^2/(cm bar)]
    beta3: float          # [fs^3/(cm bar)]
    beta4: float          # [fs^4/(cm bar)]
    n2_at_one_bar: float  # [m^2/(W bar)]
    
    def __repr__(self):
        return (f"{self.name}: beta2={self.beta2} fs^2/(cm bar), "
                f"beta3={self.beta3} fs^3/(cm bar), "
                f"beta4={self.beta4} fs^4/(cm bar), "
                f"n2_at_one_bar={self.n2_at_one_bar} m^2/(W bar)")

noble_gases = [
    NobleGasData(name="Neon", beta2=0.0202, beta3=0.0158, beta4=0, n2_at_one_bar=0.14e-23),
    NobleGasData(name="Argon", beta2=0.1980, beta3=0.1586, beta4=0, n2_at_one_bar=1.74e-23),
    NobleGasData(name="Krypton", beta2=0.3996, beta3=0.3298, beta4=0, n2_at_one_bar=4.03e-23),
    NobleGasData(name="Xenon", beta2=0.9113, beta3=0.7836, beta4=0, n2_at_one_bar=11.15e-23),
]

In [ ]:
class GasPropertyBuilder:
    
    def __init__(self, fiber_length, fiber_radius, pulse_wavelength, n2_at_one_bar, betas_at_one_bar,  
                 constant_pressure=None, pressure_boundary=None, differential_pumping=False):
        
        assert (constant_pressure is not None) if not differential_pumping else True, \
        "A constant gas pressure is expected for non-differential pumping scheme."
        assert (pressure_boundary is not None) if differential_pumping else True, \
        "A list containing the boundary values of the gas pressure at both input \
        and output surface of the fiber is expected for differential pumping scheme."
        
        self.fiber_len = fiber_length
        self.fiber_rad = fiber_radius
        self.pulseWL = pulse_wavelength
        
        if differential_pumping:
            if len(pressure_boundary) != 2:
                print("A list containing the boundary values of the gas pressure at both input \
                and output surface of the fiber is expected for differential pumping scheme.")
            self.pressure_entr = pressure_boundary[0]
            self.pressure_exit = pressure_boundary[1]
        else:
            self.const_pressure = constant_pressure
        
        self.n2_at_one_bar = n2_at_one_bar
        self.betas_at_one_bar = betas_at_one_bar
        self.differential_pumping = differential_pumping
        
    def torrToBar(self, pressure_Torr):
        """
        Unit conversion for gas pressure from torr to bar.
        
        1 torr is defined as exactly 1/760 of a standard atmosphere (1 atm = 101325 Pa), 
        1 pascal is equal to 1e-5 bar.

        Input(s):
        pressure_Torr: gas pressure [Torr]
        Output(s):
        pressure_Bar: gas pressure [Bar]
        """
        
        pressure_Pa = pressure_Torr * (101325/760)
        pressure_Bar = pressure_Pa * 1e-5
        
        return pressure_Bar

    def pressureDistribution(self, z=None):
        """
        Generate gas pressure distribution along the fiber under differential pumping.

        Reference: 
        Suda, Akira, et al.
        “Generation of Sub-10-Fs, 5-mJ-Optical Pulses Using a Hollow Fiber with a Pressure Gradient.”
        Applied Physics Letters, vol. 86, no. 11, 2005, https://doi.org/10.1063/1.1883706.
        
        Input(s):
        z: position grid along the fiber [m].
        fiber_len: length of the fiber [m].
        pressure_entr: gas pressure at the entrance [Torr].
        pressure_exit: gas pressure at the exit [Torr].
        Output(s):
        pressure_dist: gas pressure distribution along the fiber [Bar].
        """
        if self.differential_pumping:
            pressure_entr = self.torrToBar(self.pressure_entr)
            pressure_exit = self.torrToBar(self.pressure_exit)
            pressure_dist = np.sqrt(pressure_entr**2 + (z/self.fiber_len) * (pressure_exit**2 - pressure_entr**2))
        else:
            pressure_dist = self.torrToBar(self.const_pressure)

        return pressure_dist
        
    def dispersionFunction(self, z=None):
        """
        Generate the fiber dispersion that can vary as a function of the length z along the fiber 
        for differential pumping, return a tuple of beta coefficients as a function of z.
        
        Input(s):
        pressure_dist: gas pressure distribution along the fiber [Bar].
        betas_at_one_bar: a list of beta coefficients (i.e. [beta2, beta3, beta4]) specified at 1 bar [ps^n/km].
        Output(s):
        betas: a tuple of beta coefficients as a function of z [ps^n/m].
        """

        if self.differential_pumping:
            pressure_dist = self.pressureDistribution(z)
        else:
            pressure_dist = self.pressureDistribution()
            
        betas = [beta * (1/1e3) * pressure_dist for beta in self.betas_at_one_bar]
        
        return tuple(betas)
    
    def nonlinearIndexN2(self, z=None):
        """
        Calculate the nonlinear refractive index n2, return a constant for non-differential pumping, 
        or a distribution along the fiber for differential pumping.

        Input(s):
        pressure_dist: gas pressure distribution along the fiber under differential pumping [Bar].
        const_pressure: constant gas pressure for non-differential pumping [Bar].
        n2_at_one_bar: nonlinear refractive index of the gas reported at 1 bar [m^2/(W bar)].
        Output(s):
        n2: nonlinear refractive index at the specified pressure [m^2/W].
        """
        
        assert (z is None) if not self.differential_pumping else True, \
        "A position grid is not needed for non-differential pumping."
        assert (z is not None) if self.differential_pumping else True, \
        "A position grid is needed for differential pumping."
        
        if self.differential_pumping:
            pressure_dist = self.pressureDistribution(z)
        else:
            pressure_dist = self.pressureDistribution()
        
        n2 = pressure_dist * self.n2_at_one_bar
        
        return n2
        
    def gammaDistribution(self, z=None):
        """
        Calculate the nonlinear parameter Gamma using Gamma=(n2*w0)/(c*A_eff), return a constant 
        for non-differential pumping, or a distribution along the fiber for differential pumping. 
        Reference: Agrawal, Govind. Nonlinear Fiber Optics. Sixth edition., Academic Press, 2019.
        
        c: speed of light in vacuum [m/s].
        w0: angular frequency [rad/s].
        A_eff: effective core area of the fiber [m^2].

        Input(s):
        n2: nonlinear refractive index at the specified pressure [m^2/W].
        pulseWL: pulse central wavelength [nm].
        fiber_rad: radius of fiber [m].
        Output(s):
        Gamma: nonlinear parameter [1/(W m)]
        """
        
        assert (z is None) if not self.differential_pumping else True, \
        "A position grid is not needed for non-differential pumping."
        assert (z is not None) if self.differential_pumping else True, \
        "A position grid is needed for differential pumping."
        
        w0 = (2*np.pi*constants.c) / (self.pulseWL*1e-9)
        A_eff = np.pi * self.fiber_rad**2
        
        if self.differential_pumping:
            n2 = self.nonlinearIndexN2(z)
        else:
            n2 = self.nonlinearIndexN2()
            
        Gamma = (n2*w0) / (constants.c*A_eff)
        
        return Gamma


In [ ]:
def loadGasParameters(gas_name):
    
        noble_gases_dict = {gas.name.lower(): gas for gas in noble_gases}
        gas = noble_gases_dict.get(gas_name.lower(), f"Gas data not found.")
        n2_at_one_bar = gas.n2_at_one_bar
        beta2 = gas.beta2 * (1e-3)**2 / 1e-5  # [ps^2/(km bar)]
        beta3 = gas.beta3 * (1e-3)**3 / 1e-5  # [ps^3/(km bar)]
        beta4 = gas.beta4 * (1e-3)**4 / 1e-5  # [ps^4/(km bar)]
        betas_at_one_bar = [beta2, beta3, beta4]
        
        return n2_at_one_bar, betas_at_one_bar    

In [ ]:
def photonEnergyGrid(pulse):
    """
    Convert the grid corresponding to the complex electric field in frequency
    domain from angular frequency to photon energy in eV.
    
    Photon energy:
    E[J] = hbar * w = h * v
    E[eV] = E[J] / e
    
    Input(s):
    pulse: a pulse instance on which the conversion is performed.
    Output(s):
    W_eV: photon energy grid [eV].
    """
    
    W_eV = constants.h * pulse.F_mks / constants.e
    
    return W_eV

In [ ]:
def dB(num):
        return 10 * np.log10(np.abs(num)**2)

In [ ]:
def calc_FWHM(pulse):
    
    t = pulse.T_mks
    temporal_intensity = np.abs(pulse.AT)**2
    
    normalized_temporal_intensity = temporal_intensity / np.max(temporal_intensity)
    
    FWHM_filter = np.where(normalized_temporal_intensity >= 0.5)[0]
    
    if len(FWHM_filter) >= 2:
        FWHM_mks = t[FWHM_filter[-1]] - t[FWHM_filter[0]]
        return FWHM_mks
    else:
        print("Pulse does not have a well-defined FWHM.")

In [ ]:
class PeakIntensityBuilder:
    
    def __init__(self, SHG_efficiency, pulse_duration, peak_intensity_CGS=None):
        
        self.eta = SHG_efficiency
        self.FWHM = pulse_duration                                # [s]
        self.peak_intensity_CGS = peak_intensity_CGS              # [W/cm^2]
        self.peak_intensity_MKS = self.toMKS(peak_intensity_CGS)  # [W/m^2]
    
    def toMKS(self, intensity_CGS):
        """
        Unit conversion for optical intensity from W/cm^2 to W/m^2.
        
        Input(s):
        intensity_CGS: intensity [W/cm^2]
        Output(s):
        intensity_MKS: intensity [W/m^2]
        """
        intensity_MKS = intensity_CGS / (1e-2)**2
        return intensity_MKS
    
    def toCGS(self, intensity_MKS):
        """
        Unit conversion for optical intensity from W/m^2 to W/cm^2.
        
        Input(s):
        intensity_MKS: intensity [W/m^2]
        Output(s):
        intensity_CGS: intensity [W/cm^2]
        """
        intensity_CGS = intensity_MKS / (1e2)**2
        return intensity_CGS
    
    def divideEPP(self, total_EPP):
        EPP_FD = total_EPP / (1 + self.eta)
        EPP_SH = self.eta * EPP_FD
        return EPP_FD, EPP_SH
    
    def calcFiberRadius(self, total_EPP_limit):
        """
        Calculate the maximum fiber radius in [m] that maintains the required peak intensity
        for a given pulse energy and pulse duration.
        """
        assert self.peak_intensity_MKS is not None, \
        "A target peak intensity must be provided to enable this function."
        EPP_FD_limit, EPP_SH_limit = self.divideEPP(total_EPP_limit)
        fiber_rad_limit = np.sqrt(EPP_SH_limit / (self.peak_intensity_MKS * self.FWHM * np.pi))
        return fiber_rad_limit
        
    def calcPeakIntensity(self, reference_EPP, fiber_radius):
        """
        Calculate the resulting peak intensity in [W/cm^2] from the given pulse energy,
        pulse duration and fiber radius.
        """
        peak_intensity_MKS = reference_EPP / (self.FWHM * np.pi * fiber_radius**2)
        peak_intensity_CGS = self.toCGS(peak_intensity_MKS)
        return peak_intensity_CGS
    
    def calcEPP(self, fiber_radius):
        """
        Calculate the pulse energies in [J] that maintain the required peak intensity
        for a given fiber radius and pulse duration.
        """
        assert self.peak_intensity_MKS is not None, \
        "A target peak intensity must be provided to enable this function."
        EPP_SH = self.peak_intensity_MKS * self.FWHM * np.pi * fiber_radius**2
        EPP_FD = EPP_SH / self.eta
        total_EPP = EPP_FD + EPP_SH
        return total_EPP, EPP_FD, EPP_SH
        

### Test for UVC plot, fixed fiber radius:

In [ ]:
# pulse parameters
FWHM = 0.03                # pulse duration [ps]
pulseWL = 800              # pulse central wavelength [nm]
SHG_efficiency = 0.2       # SHG conversion efficiency
total_EPP_limit = 1e-3     # pulse energy limit of laser source [J]
GDD = 0.0                  # group delay dispersion [ps^2]
TOD = 0.0                  # third order dispersion [ps^3]
peak_intensity_CGS = 2e12  # peak intensity for nonlinear processes [W/cm^2]

# fiber parameters
Length = 100       # fiber length [mm]
Alpha = 0.0        # attentuation coefficient [dB/cm]
fibWL = pulseWL    # center wavelength of fiber [nm]
alpha = np.log((10**(Alpha * 0.1))) * 100  # convert from dB/cm to 1/m


# simulation parameters
Window = 2      # simulation window [ps]
Steps = 100        # simulation steps
Points = 2**14    # simulation points
Raman = True      # Enable Raman effect?
Steep = True      # Enable self steepening?

intensitybuilder = PeakIntensityBuilder(SHG_efficiency=SHG_efficiency, 
                                        pulse_duration=FWHM*1e-12, 
                                        peak_intensity_CGS=peak_intensity_CGS)


In [ ]:
# largest fiber radius that support nonlinear processes for current laser parameters [m]
fiber_rad_limit = intensitybuilder.calcFiberRadius(total_EPP_limit=total_EPP_limit)
print(f'If a total pulse energy of {total_EPP_limit*1e3} mJ is available, the maximum possible fiber radius is {fiber_rad_limit*1e6:.4} microns.')

# fiber radius [m]
fiber_rad = np.array([100e-6])

# energy division
total_EPP, EPP_FD, EPP_SH = intensitybuilder.calcEPP(fiber_radius=fiber_rad)

for i in range(len(fiber_rad)):
    print(f'For fiber radius {fiber_rad[i]*1e6:.4} microns, a total of {total_EPP[i]*1e3:.4} mJ is needed, of which the fundamental pulse has {EPP_FD[i]*1e3:.4} mJ, the second harmonic pulse has {EPP_SH[i]*1e3:.4} mJ.')

# load gas parameters
n2_at_one_bar, betas_at_one_bar = loadGasParameters("Xenon")

# initialize data frames
signal_energy_data = pd.DataFrame(columns=['length', 'pressure', 'energy'])
signal_peak_power_data = pd.DataFrame(columns=['length', 'pressure', 'peak power'])
signal_photon_flux_data = pd.DataFrame(columns=['length', 'pressure', 'photon flux'])

In [ ]:
# static pressures [Torr]
pressures = np.linspace(1, 1000, 500, endpoint=True)

for i in range(len(pressures)):
    
    # build the gas properties
    gasbuilder = GasPropertyBuilder(fiber_length=Length*1e-3, fiber_radius=fiber_rad[0], pulse_wavelength=pulseWL, 
                                    betas_at_one_bar=betas_at_one_bar, constant_pressure=pressures[i], pressure_boundary=None, 
                                    n2_at_one_bar=n2_at_one_bar, differential_pumping=False)
    # create the fiber
    fiber = pynlo.media.fibers.fiber.FiberInstance()
    fiber.generate_fiber(Length * 1e-3, center_wl_nm=fibWL, betas=gasbuilder.dispersionFunction(),
                              gamma_W_m=gasbuilder.gammaDistribution(), gvd_units='ps^n/m', gain=-alpha)

    # create the fundamental pulse
    pulse_FD = pynlo.light.DerivedPulses.SechPulse(
        1, FWHM/1.76, pulseWL, time_window_ps=Window,
        GDD=GDD, TOD=TOD, NPTS=Points, frep_MHz=1e-3, power_is_avg=True
    )
    pulse_FD = pulse_FD.interpolate_to_new_center_wl(pulseWL/3)
    pulse_FD.set_epp(EPP_FD[0])
    
    # create the second harmonic pulse
    pulse_SH = pynlo.light.DerivedPulses.SechPulse(
        1, FWHM/1.76, pulseWL/2, time_window_ps=Window,
        GDD=GDD, TOD=TOD, NPTS=Points, frep_MHz=1e-3, power_is_avg=True
    )
    pulse_SH = pulse_SH.interpolate_to_new_center_wl(pulseWL/3)
    pulse_SH.set_epp(EPP_SH[0])
    
    # create the combined pulse
    pulse = pynlo.light.PulseBase.Pulse()
    pulse.set_NPTS(Points)
    pulse.set_time_window_ps(Window)
    pulse.set_center_wavelength_nm(fibWL)
    pulse.set_AW(pulse_FD.AW + pulse_SH.AW)

    # propagation
    evol = pynlo.interactions.FourWaveMixing.SSFM.SSFM(local_error=0.001, USE_SIMPLE_RAMAN=True, 
                                                       disable_Raman=np.logical_not(Raman), 
                                                       disable_self_steepening=np.logical_not(Steep))
    
    y, AW, AT, pulse_out = evol.propagate(pulse_in=pulse, fiber=fiber, n_steps=Steps, reload_fiber_each_step=False)

    if i % 50 == 0:
        
        # plot 1, keep track of the process
        fig = plt.figure(figsize=(15,10))
        ax0 = plt.subplot2grid((2,2), (0, 0), rowspan=1)
        ax1 = plt.subplot2grid((2,2), (0, 1), rowspan=1)
        ax2 = plt.subplot2grid((2,2), (1, 0), rowspan=1, sharex=ax0)
        ax3 = plt.subplot2grid((2,2), (1, 1), rowspan=1, sharex=ax1)
    
        # y_mm = y * 1e3
        W_eV = photonEnergyGrid(pulse_SH)
        
        zW = dB(np.transpose(AW)[:, (W_eV > 0)])
        zT = dB(np.transpose(AT))
    
        spectral_intensity = np.abs(np.transpose(AW)[:, (W_eV > 0)])**2
        temporal_intensity = np.abs(np.transpose(AT))**2
        
        ax0.plot(W_eV[W_eV > 0], spectral_intensity[-1], color='r', linestyle='solid', label='spectral intensity at the exit')
        ax0.plot(W_eV[W_eV > 0], spectral_intensity[0], color='b', linestyle='dashed', label='spectral intensity at the entrance')
    
        ax1.plot(pulse.T_ps, temporal_intensity[-1], color='r', label='temporal intensity at the exit')
        ax1.plot(pulse.T_ps, temporal_intensity[0], color='b', label='temporal intensity at the entrance')
    
        extent = (np.min(W_eV[W_eV > 0]), np.max(W_eV[W_eV > 0]), 0, Length)
        ax2.imshow(zW, extent=extent, vmin=np.max(zW) - 250.0,
                         vmax=np.max(zW), aspect='auto', origin='lower', cmap='jet')
        
        extent = (np.min(pulse.T_ps), np.max(pulse.T_ps), 0, Length)
        ax3.imshow(zT, extent=extent, vmin=np.max(zT) - 60.0,
                   vmax=np.max(zT), aspect='auto', origin='lower', cmap='jet')
    
        ax0.legend()
        ax0.set_yscale('log')
        ax0.set_ylabel('Spectral power density (W/eV)')
        
        ax1.legend()
        ax2.set_xlabel('Photon energy (eV)')
        ax3.set_xlabel('Time (ps)')
        
        ax2.set_xlim(0,18.5)
        ax2.set_ylabel('Propagation distance (mm)')
        
        HHGs = [1.55 * (i + 1) for i in range(11)]
        for HHG in HHGs:
            ax0.axvline(x=HHG, color='g', linestyle=':')
            
        plt.tight_layout()
        plt.show()

    # UVC signals
    lower_wl_nm = 200
    upper_wl_nm = 300
    UVC_filter = (pulse_out.wl_nm > lower_wl_nm) & (pulse_out.wl_nm < upper_wl_nm)
    
    AW_new = np.transpose(AW)
    AW_new[:, pulse_out.wl_nm < lower_wl_nm] = 0.0
    AW_new[:, pulse_out.wl_nm > upper_wl_nm] = 0.0    

    UVC_wavelength = (800/3) * 1e-9
    UVC_photon_energy = constants.h * constants.c / UVC_wavelength

    length_along_fiber = np.linspace(0, 100, AW_new.shape[0], endpoint=True)
    
    signal_energies = []
    signal_peak_powers = []
    signal_photon_fluxes = []
    
    for step in range(AW_new.shape[0]):
        
        pulse_UVC = pulse_out.create_cloned_pulse()
        pulse_UVC.set_AW(AW_new[step,:])
        
        spectral_power = np.abs(pulse_UVC.AW[W_eV>0])**2
        signal_energy = np.trapz(spectral_power) / pulse_UVC.dF_mks
        signal_peak_power = signal_energy / calc_FWHM(pulse_out)
        signal_photon_flux = signal_peak_power / UVC_photon_energy
    
        signal_energies.append(signal_energy)
        signal_peak_powers.append(signal_peak_power)
        signal_photon_fluxes.append(signal_photon_flux)

        if step % 20 ==0:

        print(f"The calculated FWHM of UVC pulse is: {calc_FWHM(pulse_UVC)*1e12} ps.")
        print(f"The calculated FWHM of output pulse is: {calc_FWHM(pulse_out)*1e12} ps.")
            
        # plot 2, check pulse shape in time domain for FWHM calculation
        fig = plt.figure(figsize=(10,5))
        ax0 = plt.subplot2grid((1,2), (0, 0), rowspan=1)
        ax1 = plt.subplot2grid((1,2), (0, 1), rowspan=1)
    
        W_eV = photonEnergyGrid(pulse_UVC)
        
        zW = dB(pulse_UVC.AW[(W_eV > 0)])
        zT = dB(pulse_UVC.AT)
    
        spectral_intensity = np.abs(pulse_UVC.AW[(W_eV > 0)])**2
        temporal_intensity = np.abs(pulse_UVC.AT)**2
        
        ax0.plot(W_eV[W_eV > 0], spectral_intensity, color='r', linestyle='solid')
        ax1.plot(pulse.T_ps, temporal_intensity, color='r')
    
        ax0.set_yscale('log')
        ax0.set_ylabel('Spectral power density (W/eV)')
        
        ax0.set_xlim(0,18.5)
        
        HHGs = [1.55 * (i + 1) for i in range(11)]
        for HHG in HHGs:
            ax0.axvline(x=HHG, color='g', linestyle=':')
            
        plt.tight_layout()
        plt.show()
        
    if i % 10 ==0:
        
        # plot 3, check integrated signals
        fig = plt.figure(figsize=(5,12.5))
        ax0 = plt.subplot2grid((3,1), (0, 0), rowspan=1)
        ax1 = plt.subplot2grid((3,1), (1, 0), rowspan=1, sharex=ax0)
        ax2 = plt.subplot2grid((3,1), (2, 0), rowspan=1, sharex=ax0)
    
        ax0.plot(length_along_fiber, signal_energies)
        ax0.set_xlabel('Distance along the fiber (mm)')
        ax0.set_ylabel('Integrated UVC signal energy (J)')
    
        ax1.plot(length_along_fiber, signal_peak_powers)
        ax1.set_ylabel('Integrated UVC signal peak intensity (W)')
    
        ax2.plot(length_along_fiber, signal_photon_fluxes)
        ax2.set_ylabel('Integrated UVC signal photon flux (ph/s)')
    
        plt.tight_layout()
        plt.show()

    
    temp_signal_energy_data = pd.DataFrame({
        'length': length_along_fiber,
        'pressure': [pressures[i]] * len(length_along_fiber),
        'energy': signal_energies
    })
    temp_signal_peak_power_data = pd.DataFrame({
        'length': length_along_fiber,
        'pressure': [pressures[i]] * len(length_along_fiber),
        'peak power': signal_peak_powers,
    })
    temp_signal_photon_flux_data = pd.DataFrame({
        'length': length_along_fiber,
        'pressure': [pressures[i]] * len(length_along_fiber),
        'photon flux': signal_photon_fluxes
    })

    signal_energy_data = pd.concat([signal_energy_data, temp_signal_energy_data], ignore_index=True)
    signal_peak_power_data = pd.concat([signal_peak_power_data, temp_signal_peak_power_data], ignore_index=True)
    signal_photon_flux_data = pd.concat([signal_photon_flux_data, temp_signal_photon_flux_data], ignore_index=True)


In [ ]:
signal_energy_data.to_csv('signal_energy_fiber_radius_100microns_Xenon.csv', index=False)
signal_peak_power_data.to_csv('signal_peak_power_fiber_radius_100microns_Xenon.csv', index=False)
signal_photon_flux_data.to_csv('signal_photon_flux_fiber_radius_100microns_Xenon.csv', index=False)

In [ ]:
heatmap_signal_energy = signal_energy_data.pivot(index='pressure', columns='length', values='energy')

plt.figure(figsize=(10, 6))
plt.imshow(heatmap_signal_energy, aspect='auto', cmap='YlOrRd', origin='lower')

plt.colorbar(label='Signal Energy [J]')
plt.title('Heatmap of Integrated Signal Energy along Fiber Length for Different Pressures')
plt.xlabel('Length along Fiber')
plt.ylabel('Pressure')
plt.yticks(ticks=np.arange(len(heatmap_signal_energy.index)), labels=heatmap_signal_energy.index)

plt.show()